# Bioinformatics workflow exercise: SoS and linear mixed model

Author: Haoyue Shuai, Nov 17, 2020

This tutorial introduces a workflow language, Script of Scripts (SoS), for bioinformatics analysis pipelines, with an example implementation of various linear mixed model methods for genetic association studies.

This is an SoS Notebook with SoS kernel cells containing workflow steps written in SoS, and bash kernel cells to run these workflow steps. Please run bash codes here directly in this notebook so the output will be saved to the notebook.

## Jupyter Lab setup

Download this notebook and launch it with [JupyterLab](https://jupyter.org/). You can follow [these suggested setup instructions](http://statgen.us/lab-wiki/orientation/jupyter-setup.html).

Please first making sure you have all the kernels needed. They should be available after all software are installed as instructed:

In [10]:
jupyter kernelspec list

Available kernels:
  ir              /Users/hyeonjukim/Library/Jupyter/kernels/ir
  julia-1.0       /Users/hyeonjukim/Library/Jupyter/kernels/julia-1.0
  julia-1.5       /Users/hyeonjukim/Library/Jupyter/kernels/julia-1.5
  calysto_bash    /Users/hyeonjukim/opt/miniconda3/share/jupyter/kernels/calysto_bash
  python3         /Users/hyeonjukim/opt/miniconda3/share/jupyter/kernels/python3
  sos             /Users/hyeonjukim/opt/miniconda3/share/jupyter/kernels/sos
  python2         /usr/local/share/jupyter/kernels/python2



In [14]:
[global]
# parameter 1
parameter: n = 1.0
# parameter 2
parameter: beta = [1.0,2.0,3.0]

In [25]:
# Print the value of n with bash
[print_n]
bash: expand = '${ }'
    echo ${n}

In [26]:
sos run orientation-hkim.ipynb print_n

INFO: Running print_n: Print the value of n with bash
1.0
INFO: print_n is completed.
INFO: Workflow print_n (ID=w4bcbb8958466f710) is executed successfully with 1 completed step.



In [14]:
sos run orientation-hkim.ipynb print_n --n 666

INFO: Running print_n: Print the value of n with bash
666.0
INFO: print_n is completed.
INFO: Workflow print_n (ID=we094e7d433abb2ad) is executed successfully with 1 completed step.



In [15]:
[print_beta]
bash: expand = '${ }'
    echo ${beta}

In [16]:
sos run orientation-hkim.ipynb print_beta

INFO: Running print_beta: 
[1.0, 2.0, 3.0]
INFO: print_beta is completed.
INFO: Workflow print_beta (ID=w78fa93e094c77376) is executed successfully with 1 completed step.



In [17]:
# Print log(beta) with Python
[log_beta]
python: expand = '${ }'
    import numpy as np
    print(np.log(${beta}))

In [18]:
sos run orientation-hkim.ipynb log_beta

INFO: Running log_beta: Print log(beta) with Python
[0.         0.69314718 1.09861229]
INFO: log_beta is completed.
INFO: Workflow log_beta (ID=w077f5b194f70ad1b) is executed successfully with 1 completed step.



In [19]:
# Print exp(n) with R
[exp_n]
R: expand = '${ }'
    print(exp(${n}))

In [20]:
sos run orientation-hkim.ipynb exp_n

INFO: Running exp_n: Print exp(n) with R
[1] 2.718282
INFO: exp_n is completed.
INFO: Workflow exp_n (ID=w68dec66676a24f9f) is executed successfully with 1 completed step.



In [21]:
sos run orientation-hkim.ipynb -h

usage: sos run orientation-hkim.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  print_n
  print_beta
  log_beta
  exp_n

Global Workflow Options:
  --n 1.0 (as float)
                        parameter 1
  --beta 1.0 2.0 3.0 (as list)
                        parameter 2

Sections
  print_n:              Print the value of n with bash
  print_beta:
  log_beta:             Print log(beta) with Python
  exp_n:                Print exp(n) with R



In [7]:
sos run ../workflow/LMM.ipynb -h

usage: sos run ../workflow/LMM.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  boltlmm
  gcta
  fastGWA
  regenie
  SAIGE

Global Workflow Options:
  --cwd VAL (as path, required)
                        the output directory for generated files
  --sampleFile VAL (as path, required)
                        Path to sample file
  --bfile VAL (as path, required)
                        Genotype files in plink binary this is used for
                        computing the GRM
  --genoFile  paths

                        Path to bgen or bed files
  --phenoFile VAL (as path, required)
                        Phenotype file for quantitative trait (BMI)
  --phenoCol VAL VA

In [24]:
cd ../data/LMM_MWE
ls

LDSCORE.1000G_EUR.tab.gz		genotypes21_22.fam
boltlmm_template.yml			imputed_genotypes.sample
fastGWA_template.yml			imputed_genotypes_chr21.bgen
genetic_map_hg19_withX.txt.gz		imputed_genotypes_chr21.bgen.bgi
genotype_inventory.txt			imputed_genotypes_chr22.bgen
genotypes.bed				imputed_genotypes_chr22.bgen.bgi
genotypes.bim				phenotypes.txt
genotypes.fam				regenie_template.yml
genotypes21_22.bed			regions.txt
genotypes21_22.bim			unrelated_samples.txt



In [29]:
cd ~/GIT/orientation/notebook/

In [31]:
sos run ../workflow/LMM.ipynb fastGWA \
    --cwd ../data/LMM_MWE/output \
    --bfile ../data/LMM_MWE/genotypes.bed \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/fastGWA_template.yml \
    --phenoCol BMI \
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 1 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --parts 2 \
    --p-filter 1

INFO: Running fastGWA_1: fastGWA mixed model (based on the sparse GRM generated above)
INFO: Running gcta_1: Partition the GRM into 100 parts and allocate 8GB memory to each job
HINT: Pulling docker image statisticalgenetics/lmm:1.6
HINT: Docker image statisticalgenetics/lmm:1.6 is now up to date
INFO: gcta_1 (index=0) is completed.
INFO: gcta_1 (index=1) is completed.
INFO: gcta_1 output:   ../data/LMM_MWE/output/cache/genotypes.part_2_1.grm.bin ../data/LMM_MWE/output/cache/genotypes.part_2_1.grm.N.bin... (6 items in 2 groups)
INFO: Running gcta_2: Merge all the parts together (Linux, Mac)
INFO: gcta_2 is completed.
INFO: gcta_2 output:   ../data/LMM_MWE/output/genotypes.grm.bin ../data/LMM_MWE/output/genotypes.grm.N.bin... (3 items)
INFO: Running gcta_3: Make a sparse GRM from the merged full-dense GRM
INFO: gcta_3 is completed.
INFO: gcta_3 output:   ../data/LMM_MWE/output/genotypes.grm.sp
INFO: fastGWA_1 (index=0) is completed.
INFO: fastGWA_1 (index=1) is completed.
INFO: fastGWA_

In [38]:
%preview ../data/LMM_MWE/output/phenotypes_BMI.fastGWA.manhattan.png

In [4]:
sos run ../workflow/LMM.ipynb regenie \
    --cwd ../data/LMM_MWE/output_regenie \
    --bfile ../data/LMM_MWE/genotypes21_22.bed \
    --maf-filter 0.001 \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/regenie_template.yml \
    --phenoCol ASTHMA T2D\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 4\
    --bsize 1000 \
    --lowmem_prefix ../data/LMM_MWE/output_regenie\
    --trait bt \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p \
    --p-filter 1 \
   

Continuation prompt found - input was incomplete:
sos run ../workflow/LMM.ipynb regenie \
    --cwd ../data/LMM_MWE/output_regenie \
    --bfile ../data/LMM_MWE/genotypes21_22.bed \
    --maf-filter 0.001 \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/regenie_template.yml \
    --phenoCol ASTHMA T2D\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 4\
    --bsize 1000 \
    --lowmem_prefix ../data/LMM_MWE/output_regenie\
    --trait bt \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p \
    --p-filter 1 \


In [37]:
sos run ../workflow/LMM1.ipynb regenie_2 \
    --cwd ../data/LMM_MWE/output_regenie \
    --bfile ../data/LMM_MWE/genotypes21_22.bed \
    --maf-filter 0.001 \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/regenie_template.yml \
    --phenoCol ASTHMA T2D\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 4\
    --bsize 1000 \
    --lowmem_prefix ../data/LMM_MWE/output_regenie\
    --trait bt \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p \
    --p-filter 1 \
   

Continuation prompt found - input was incomplete:
sos run ../workflow/LMM1.ipynb regenie_2 \
    --cwd ../data/LMM_MWE/output_regenie \
    --bfile ../data/LMM_MWE/genotypes21_22.bed \
    --maf-filter 0.001 \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/regenie_template.yml \
    --phenoCol ASTHMA T2D\
    --covarCol SEX \
    --qCovarCol AGE \
    --numThreads 4\
    --bsize 1000 \
    --lowmem_prefix ../data/LMM_MWE/output_regenie\
    --trait bt \
    --minMAC 4 \
    --bgenMinMAF 0.05 \
    --bgenMinINFO 0.8 \
    --reverse_log_p \
    --p-filter 1 \


In [8]:
sos run ../workflow/LMM1.ipynb boltlmm \
    --cwd ./output_boltlmm \
    --bfile ../data/LMM_MWE/genotypes.bed \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/boltlmm_template.yml \
    --LDscoresFile ../data/LMM_MWE/LDSCORE.1000G_EUR.tab.gz \
    --geneticMapFile ../data/LMM_MWE/genetic_map_hg19_withX.txt.gz \
    --phenoCol BMI \
    --covarCol SEX \
    --covarMaxLevels 10 \
    --qCovarCol AGE \
    --numThreads 5 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --lmm-option none \
    --p-filter 1 \
 

Continuation prompt found - input was incomplete:
sos run ../workflow/LMM.ipynb boltlmm \
    --cwd ./output_boltlmm \
    --bfile ../data/LMM_MWE/genotypes.bed \
    --sampleFile ../data/LMM_MWE/imputed_genotypes.sample \
    --genoFile ../data/LMM_MWE/imputed_genotypes_chr*.bgen \
    --phenoFile ../data/LMM_MWE/phenotypes.txt \
    --formatFile ../data/LMM_MWE/boltlmm_template.yml \
    --LDscoresFile ../data/LMM_MWE/LDSCORE.1000G_EUR.tab.gz \
    --geneticMapFile ../data/LMM_MWE/genetic_map_hg19_withX.txt.gz \
    --phenoCol BMI \
    --covarCol SEX \
    --covarMaxLevels 10 \
    --qCovarCol AGE \
    --numThreads 5 \
    --bgenMinMAF 0.001 \
    --bgenMinINFO 0.1 \
    --lmm-option none \
    --p-filter 1 \


In [5]:
pwd

/Users/hyeonjukim/GIT/orientation/notebook

